In [23]:
import pandas as pd
import numpy as np
import wrds

# Imports

In [59]:
# file imports
file_path_forecasts = '../data/dataset2014-2022-v4.zip'
file_path_company = '../data/dataset-company-industry-data-v1.csv'

# IBES forecasts
df_forecasts = pd.read_csv(file_path_forecasts)
# SIC codes
df_companies = pd.read_csv(file_path_company)

In [ ]:
# setup WRDS database connection for linking table
db = wrds.Connection()

In [ ]:
# import linking information table
link_table = db.get_table(library = "comp", table = "security")
link_table = link_table[["gvkey", "iid", "ibtic"]]

# Preprocessing
df_forecasts:
- Lower case all columns in df_forecasts
- Drop unncessary columns
- Include only forecasts issued no earlier than 1 year ahead and no later than 30 days before fiscal year end
- Rename columns

In [61]:
df_forecasts = df_forecasts.rename(columns={col: col.lower() for col in df_forecasts.columns})

# Drop unncessary columns
columns_to_drop = ["fpi","measure","cusip"]
df_forecasts = df_forecasts.drop(columns=columns_to_drop)

KeyError: "['fpi', 'measure'] not found in axis"

In [62]:
# convert columns with dates to datetime
def convert_to_datetime(df):
    columns_to_convert = ['fpedats','revdats', "anndats", 'anndats_act']
    for column in columns_to_convert:
        df[column] = pd.to_datetime(df[column])
    return df
df_forecasts = convert_to_datetime(df_forecasts)

# filter forecasts  based on time
df_forecasts['difference_date'] = df_forecasts['fpedats'] - df_forecasts['anndats']
df_forecasts = df_forecasts[(df_forecasts['difference_date']> pd.Timedelta(days=30))&(df_forecasts['difference_date']<pd.Timedelta(days=365))]
df_forecasts.head()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\mariu\mambaforge\envs\learn-env\Lib\site-packages\IPython\core\formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mariu\mambaforge\envs\learn-env\Lib\site-packages\IPython\core\formatters.py", line 344, in __call__
    return method()
           ^^^^^^^^
  File "c:\Users\mariu\mambaforge\envs\learn-env\Lib\site-packages\pandas\core\frame.py", line 1178, in _repr_html_
    d.to_string(buf=buf)
               ^^^^^^^^^^
  File "c:\Users\mariu\mambaforge\envs\learn-env\Lib\site-packages\pandas\io\formats\format.py", line 1074, in to_html
  File "c:\Users\mariu\mambaforge\envs\learn-env\Lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
            ^^^^^^^^^^^^^
  File "c:\Users\mariu\mambaforge\envs\learn-env\Lib\site-packages\pandas\io\formats\html.py", line 644, in render
    super().render()
  File "c:\U

  ticker     cusip oftic           cname  estimator  analys  value    fpedats  \
1   0000  87482X10  TLMR  TALMER BANCORP       2500   72481   0.85 2015-12-31   
2   0000  87482X10  TLMR  TALMER BANCORP        149  119962   0.86 2015-12-31   
3   0000  87482X10  TLMR  TALMER BANCORP       1267   71182   0.89 2015-12-31   
4   0000  87482X10  TLMR  TALMER BANCORP        149  119962   0.89 2015-12-31   
5   0000  87482X10  TLMR  TALMER BANCORP        873   79092   0.91 2015-12-31   

     revdats    anndats  actual anndats_act difference_date  
1 2015-02-18 2015-02-18    1.02  2016-01-26        316 days  
2 2015-02-18 2015-02-18    1.02  2016-01-26        316 days  
3 2015-04-30 2015-02-18    1.02  2016-01-26        316 days  
4 2015-04-30 2015-02-23    1.02  2016-01-26        311 days  
5 2015-07-06 2015-04-07    1.02  2016-01-26        268 days  

In [64]:
# rename columns
df_forecasts = df_forecasts.rename(columns={"ticker": "ibes_ticker_pk", "oftic": "official_ticker" "analys": "analyst", 
                                  "value": "estimated_eps", "fpedats": "fiscal_period_ending", 
                                  "revdats": "revision_date", "annsdats": "announce_date", 
                                  "actual": "actual_eps", "annsdats_act": "announce_date_actual"})

## Merging

In [54]:
merged_df = pd.merge(left = df_forecasts, right = link_table, how = "left", left_on="ticker", right_on= "ibtic")
merged_df = merged_df.drop(columns=["ibtic"])

In [46]:
sic_codes = df_companies[["gvkey", "sic"]].drop_duplicates().reset_index(drop=True)

In [55]:
merged_df['gvkey'] = merged_df['gvkey'].astype(str)
sic_codes['gvkey'] = sic_codes['gvkey'].astype(str)

In [56]:
merged_df_sic = pd.merge(left=merged_df, right = sic_codes, how ="left", left_on = "gvkey", right_on="gvkey")

In [53]:
num_na_values = sic_codes['sic'].isna().sum()
print(f"The number of NA values in the 'sic' column is: {num_na_values}")

The number of NA values in the 'sic' column is: 1
